In [1]:
from __future__ import print_function

%matplotlib inline
import os
import random

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data

import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from datetime import datetime, timedelta
from IPython.display import HTML

from model import Classifier

In [2]:
# Set random seed for reproducibility
manualSeed = 999

In [3]:
random.seed(manualSeed)
torch.manual_seed(manualSeed)

## Hyperparameters

In [4]:
restore_session = False

# Dataset and dataloader
dataroot = "data/airbnb"
image_size = 256
workers = 2
batch_size = 64

# Training parameters
n_epochs = 100
lr = 0.001
ngpu = 1

In [5]:
dataset = dset.ImageFolder(root=dataroot, transform=transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

In [6]:
print('CUDA: {}'.format(torch.cuda.is_available()))
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")
run_cuda = torch.cuda.is_available() and ngpu > 0 

CUDA: False


In [7]:
fixed_images, fixed_labels = next(iter(dataloader))
fixed_images = fixed_images[:10]

In [8]:
def plot_class_progress(predictions):
    classes = np.array([str(i) for i in range(11)])

    images = fixed_images.numpy()
    labels = fixed_labels.numpy()
    predictions = predictions.numpy()

    fig, axs = plt.subplots(len(images), 2, figsize=(4, 18))

    for i, image in enumerate(images):
        pred_ind = np.argsort(predictions[i])[-3:]
        pred_disp = predictions[i][pred_ind]
        class_disp = classes[pred_ind]
        true_rating = labels[i]

        pred_disp = np.exp(pred_disp) / sum(np.exp(pred_disp))

        # axs[i][0].set_title('True rating: {}'.format(true_rating))
        image = (np.transpose(image, (1, 2, 0)) / 2) + 0.5
        axs[i][0].imshow((image * 255).astype(np.uint8))
        # axs[i][0].imshow(image[0])

        bars = axs[i][1].barh(class_disp, pred_disp, color='r')

        if str(true_rating) in class_disp:
            ind = np.where(class_disp == str(true_rating))[0][0]
            bars[ind].set_color('g')

        asp = np.diff(axs[i][1].get_xlim())[0] / np.diff(axs[i][1].get_ylim())[0]
        axs[i][1].set_aspect(asp)
    plt.savefig('class_progress.jpg')

In [9]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [10]:
classifier = Classifier().to(device)

if (device.type == 'cuda') and (ngpu > 1):
    classifier = nn.DataParallel(classifier, list(range(ngpu)))
classifier.apply(weights_init)

Classifier(
  (conv1): Conv2d(3, 32, kernel_size=(6, 6), stride=(4, 4), padding=(2, 2))
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
  (conv3): Conv2d(64, 128, kernel_size=(4, 4), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (lin1): Linear(in_features=2048, out_features=1024, bias=True)
  (lin2): Linear(in_features=1024, out_features=256, bias=True)
  (lin3): Linear(in_features=256, out_features=64, bias=True)
  (lin4): Linear(in_features=64, out_features=11, bias=True)
)

### Session Management

In [11]:
def save_session(cur_epoch, cur_iters, cur_time, loss, progress):
    model_state = { 'state_dict': classifier.state_dict(), 'optimizer': optimizer.state_dict() }
    log_state = { 'iters': cur_iters, 'epoch': cur_epoch + 1, 'runtime': cur_time,
                    'loss': loss, 'progress': progress }
    state = { 'model_state': model_state, 'log_state': log_state }
    torch.save(state, 'checkpoints/latest.tar')

In [12]:
def load_session(filename='checkpoints/latest.tar'):
    if os.path.isfile(filename):
        print('=> Loading checkpoint from {}'.format(filename))
        checkpoint = torch.load(filename)
        return checkpoint
    else:
        print('=> Checkpoint {} not found'.format(filename))

### Training

In [13]:
def print_progress(line):
    with open('out.txt', 'w') as file:
        file.write(line)

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=lr)

In [15]:
class_progress = []
loss_history = []
iters, start_epoch = 0, 0

runtime = ''
start_time = datetime.now()

In [16]:
if os.path.isfile('checkpoints/latest.tar') and restore_session:
    session = load_session('checkpoints/latest.tar')

    # Load logs
    loss_history = session['log_state']['loss']
    class_progress = session['log_state']['progress']
    iters, start_epoch = session['log_state']['iters'], session['log_state']['epoch']
    runtime = session['log_state']['runtime']

    # Load model
    classifier.load_state_dict(session['model_state']['state_dict'])
    optimizer.load_state_dict(session['model_state']['optimizer'])

    dt = datetime.strptime(runtime, '%H:%M:%S')
    runtime_delta = timedelta(hours=dt.hour, minutes=dt.minute, seconds=dt.second)
    start_time = start_time - runtime_delta

In [17]:
for epoch in range(start_epoch, n_epochs):
    for i, data in enumerate(dataloader):
        inputs, labels = data
        optimizer.zero_grad()

        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = classifier(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        running_loss = loss.item()
        loss_history.append(running_loss)

        runtime = str(datetime.now() - start_time).split('.')[:1][0]

        print_progress('Runtime: \e[32m %s \e[0m\n\n[\e[32m%d\e[0m/\e[32m%d\e[0m] [\e[32m%d\e[0m/\e[32m%d\e[0m] [\e[32m%d\e[0m] loss: \e[32m%.4f\e[0m CUDA: \e[32m%s\e[0m' %
            (runtime, epoch + 1, n_epochs, i, len(dataloader), iters, running_loss, run_cuda))

        if i % 10 == 0:
            with torch.no_grad():
                out = classifier(fixed_images.to(device)).cpu()
            plot_class_progress(out)
            class_progress.append(out)
        
        if iters % 1000 is 0 or (epoch is n_epochs - 1 and i is len(dataloader) - 1):
            save_session(epoch, i, runtime, loss_history, class_progress)
    
        iters += 1

KeyboardInterrupt: 